# Jane Street Market Prediction 
> Buy low, sell high. It sounds so easy….

- toc: true
- badges: true
- author: Austin Chen
- categories: [time series,stock,kaggle]

# 1% Better Everyday

- I see that autoencoders are widely being used in this competition. However, I am having some doubts on wether the autoencoded features really help the model. As shown in the notebook, the compressed features concatenated back into the original data as extra featues. For me personally, I've tried 2~3 different random seed and the variation on the results are large. It makes me wonder that the autoencoder is only useful on certain features, so dropping these from the autoencoder might be a good idea.

- Maybe we can feed the autoencoded features to non-nn models e.g. Xgboost, CatBoost, CBM? [refence](https://www.kaggle.com/gregorycalvez/de-anonymization-time-aggregation-tags)

- Maybe we need to remove correlated featues before generating autoencoded features.


# Overview

The efficient market hypothesis posits that markets cannot be beaten because asset prices will always reflect the fundamental value of the assets. In a perfectly efficient market, buyers and sellers would have all the agency and information needed to make rational trading decisions.

In reality, financial markets are not efficient. The purpose of this trading model is to identify arbitrage opportunities to "buy low and seell high". In other words, we exploit market inefficiencies to identify and decide whether to execute profitable trades.

The dataset, provided by Jane Street, contains an anonymized set of 129 features representing real stock market data. Each row in the dataset represents a trading opportunity, for which I predict an action value: 1 to amke the trade and 0 to pass on it. Due to the high demensionality of the dataset, I use Principal Components Analysis (PCA) to identify features to be used for supervised learning. The intuition is to compress the dataset and use it more efficiently. I then use XGBoost (extreme gradient boosting) - a hugely popular ML library due to its superior execution speed and model performance - to predict profitable trades. I also use Optuna (an automatic hyperparameter optimization software framework) to tune the hyperparameters of the classification model.

# Prework

In [2]:
#hide

%%writefile conditional_cell_extension.py
def run_if(line, cell=None):
    '''Execute current line/cell if line evaluates to True.'''
    if not eval(line):
        return
    get_ipython().ex(cell)

def load_ipython_extension(shell):
    '''Registers the run_if magic when the extension loads.'''
    shell.register_magic_function(run_if, 'line_cell')

def unload_ipython_extension(shell):
    '''Unregisters the run_if magic when the extension unloads.'''
    del shell.magics_manager.magics['cell']['run_if']

Overwriting conditional_cell_extension.py


In [3]:
#hide
%reload_ext conditional_cell_extension

In [4]:
#hide
!pip install dabl > /dev/null
!pip install datatable > /dev/null
!pip install keras-tuner > /dev/null

In [5]:
#collapse-hide
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt
import os, gc, cv2, random, warnings
import re, math, sys, json, pprint, pdb

import tensorflow as tf
from tensorflow.keras import backend as K
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

import dabl
import datatable as dt
import kerastuner as kt

warnings.simplefilter('ignore')
print(f"Using TensorFlow v{tf.__version__}")

Using TensorFlow v2.4.0


In [6]:
#hide
#@title Accelerator { run: "auto" }
DEVICE = 'GPU' #@param ["None", "'GPU'", "'TPU'"] {type:"raw", allow-input: true}

if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTOTUNE = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

Using default strategy for CPU and single GPU
Num GPUs Available:  1
REPLICAS: 1


In [ ]:
#hide
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
                name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [8]:
#collapse-hide
#@title Notebook type { run: "auto", display-mode:"form" }
SEED = 10120919
DEBUG = False #@param {type:"boolean"}
TRAIN = True #@param {type:"boolean"}

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

GOOGLE = 'google.colab' in str(get_ipython())
KAGGLE = not GOOGLE

seed_everything(SEED)

print("Running on {}!".format(
   "Google Colab" if GOOGLE else "Kaggle Kernel"
))

Running on Google Colab!


In [9]:
#hide
%%run_if {GOOGLE}
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [10]:
#hide
project_name = 'jane-street-market-prediction'
root_path  = '/content/gdrive/MyDrive/' if GOOGLE else '/'
input_path = f'{root_path}kaggle/input/{project_name}/'
working_path = f'{input_path}working/' if GOOGLE else '/kaggle/working/'
os.makedirs(working_path, exist_ok=True)
os.chdir(working_path)
os.listdir(input_path)

['train.csv',
 'working',
 '__init__.py',
 'competition.cpython-37m-x86_64-linux-gnu.so',
 'train.csv.zip',
 'example_sample_submission.csv',
 'features.csv',
 'example_test.csv.zip']

In [ ]:
#hide
!kaggle competitions download -c jane-street-market-prediction

# Data

## Loading

Let's try [`datatable`](https://github.com/h2oai/datatable) to load the data, and then convert to a pandas dataframe.

In [ ]:
%%run_if {}
train_dt = dt.fread(f"{input_path}train.csv")

CPU times: user 30.9 s, sys: 6.73 s, total: 37.6 s
Wall time: 3min 20s


In [ ]:
train_df = train_dt.to_pandas()

CPU times: user 5.35 s, sys: 4.15 s, total: 9.5 s
Wall time: 7.2 s


The dataframe is `2390490` rows by `138` columns

In [ ]:
#hide_input
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float64(135), int32(3)
memory usage: 2.4 GB


## Exploration

This dataset coantians an anonymized set of features, featue_{0..129}, representing real stock market data. Each row in the dataset represents a trading opportunity, for which which you will be predicting an action value: 1 to make the trade and 0 to pass on it. Each trade has an associated weight and resp, which together represents a return on the trade. the data column is an integer which represents the day of the trade, while `ts_id` represents a time ordering. In addition to anonymized feature values, you are provided iwth metatdata about the features in `features.csv`{% fn 1%}

In the training set, `train.csv`, you are provided a `resp` value, as well as several other `resp_{1,2,3,4}` values that represent returns over different time horizons. These variables are not included in the test set. 

{{ 'Kaggle Notebook: [EDA / A Quant's Prespective](https://www.kaggle.com/hamzashabbirbhatti/eda-a-quant-s-prespective) @hamzashabbirbhatti` | fndetail: 1 }}

In [13]:
feature_df = pd.read_csv(f"{input_path}features.csv")

In [16]:
feature_df.head()

,feature,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15,tag_16,tag_17,tag_18,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28
0,feature_0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,feature_1,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,feature_2,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,feature_3,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,feature_4,False,False,False,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Preparing

In [ ]:
train_df = train_df.query('date > 85').reset_index(drop=True)
# limit memory usage
train_df = train_df.astype({c: np.float32
    for c in train_df.select_dtypes(include='float64').columns})
train_df.fillna(train_df.mean(), inplace=True)
train_df = train_df.query('weight > 0').reset_index(drop = True)

train_df['action'] = ((train_df['resp_1'] > 0) &
                      (train_df['resp_2'] > 0) &
                      (train_df['resp_3'] > 0) &
                      (train_df['resp_4'] > 0) &
                      (train_df['resp'] > 0)).astype('int')

features = [c for c in train_df.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

x_train = train_df[features].values
y_train = np.stack([(train_df[col] > 0).astype('int') 
                        for col in resp_cols]).T
                        
f_mean = np.mean(train_df[features[1:]].values, axis=0)

> Note: Modified code for [`class GroupTimeSeriesSplit(_BaseKFold)`](https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243)

In [ ]:
#collapse-show

from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
#collapse-show
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

# Model

As many people have mentioned, finance datasets are usually have low signal-to-noise ratio (SNR). So, here comes an intuitive question. Does Denoising Auto Encoder (DAE) help in reducing noise and cleaning data in finance datasets?

The idea of using an encoder is the denoise the data. After many attempts at using a unsupervised autoencoder, the choice landed on a bottleneck encoder as this will preserve the intra-feature relations.

## Building the autoencoder

The autoencoder should aid in denoising the data based on [this](https://www.semanticscholar.org/paper/Deep-Bottleneck-Classifiers-in-Supervised-Dimension-Parviainen/fb86483f7573f6430fe4597432b0cd3e34b16e43) paper.

In [ ]:
def build_autoencoder(input_dim, output_dim, noise=.05):
    inputs = tf.keras.layers.Input(input_dim)
    encoded = tf.keras.layers.BatchNormalization()(inputs)
    encoded = tf.keras.layers.GaussianNoise(noise)(encoded)
    encoded = tf.keras.layers.Dense(640, activation='relu')(encoded)
    decoded = tf.keras.layers.Dropout(0.2)(encoded)
    decoded = tf.keras.layers.Dense(input_dim, name='decoded')(decoded)
    x = tf.keras.layers.Dense(320, activation='relu')(decoded)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(output_dim, activation='sigmoid',
                              name='label_output')(x)

    encoder = tf.keras.models.Model(inputs=inputs, outputs=encoded)

    autoencoder = tf.keras.models.Model(inputs=inputs, outputs=[decoded,x])
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                        loss={'decoded':'mse',
                              'label_output':'binary_crossentropy'})
    return autoencoder, encoder

## Building the MLP

In [ ]:
def build_model(hp, input_dim, output_dim, encoder):
    inputs = tf.keras.layers.Input(input_dim)
    
    x = encoder(inputs)
    
    x = tf.keras.layers.Concatenate()([x,inputs]) #use both raw and encoded features
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(hp.Float('init_dropout',0.0,0.5))(x)
    
    for i in range(hp.Int('num_layers',1,5)):
        x = tf.keras.layers.Dense(hp.Int('num_units_{i}',128,256))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Lambda(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(hp.Float(f'dropout_{i}',0.0,0.5))(x)

    x = tf.keras.layers.Dense(output_dim,activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=x)

    model.compile(optimizer = tf.keras.optimizers.Adam(
                    hp.Float('lr',0.00001,0.1,default=0.001)),
                  loss = tf.keras.losses.BinaryCrossentropy(
                    label_smoothing = hp.Float('label_smoothing',0.0,0.1)),
                  metrics = [tf.keras.metrics.AUC(name = 'auc')])
    return model

## Defining and training the autoencoder

We add gaussian noise with mean and std from training datea. After training we lock the layersfin the encoder from further training.

In [ ]:
autoencoder, encoder = build_autoencoder(x_train.shape[-1], 
                                         y_train.shape[-1],
                                         noise=0.1)

In [ ]:
#collapse-output
autoencoder.fit(x_train,(x_train, y_train),
                epochs=1000,
                batch_size=4096, 
                validation_split=0.1,
                callbacks = [
                    tf.keras.callbacks.EarlyStopping(
                        'val_loss', patience=10,
                        restore_best_weights=True)])

Epoch 1/1000
346/346 [==============================] - 7s 12ms/step - loss: 2.1184 - decoded_loss: 1.3832 - label_output_loss: 0.7352 - val_loss: 0.8046 - val_decoded_loss: 0.1152 - val_label_output_loss: 0.6894
Epoch 2/1000
346/346 [==============================] - 3s 10ms/step - loss: 1.0088 - decoded_loss: 0.3173 - label_output_loss: 0.6915 - val_loss: 0.7703 - val_decoded_loss: 0.0810 - val_label_output_loss: 0.6893
Epoch 3/1000
346/346 [==============================] - 3s 10ms/step - loss: 0.9952 - decoded_loss: 0.3053 - label_output_loss: 0.6899 - val_loss: 0.7617 - val_decoded_loss: 0.0728 - val_label_output_loss: 0.6890
Epoch 4/1000
346/346 [==============================] - 3s 9ms/step - loss: 0.9690 - decoded_loss: 0.2794 - label_output_loss: 0.6895 - val_loss: 0.7576 - val_decoded_loss: 0.0686 - val_label_output_loss: 0.6890
Epoch 5/1000
346/346 [==============================] - 3s 9ms/step - loss: 0.9750 - decoded_loss: 0.2856 - label_output_loss: 0.6894 - val_loss: 0.7

In [ ]:
encoder.save_weights('encoder.hdf5')

In [ ]:
encoder.load_weights('encoder.hdf5')
encoder.trainable = False

## Running CV

Following [this notebook](https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv) which use 5 `PurgedGroupTimeSeriesSplit` split on the dates in the training data. 

We add the locked encoder as the first layer of the MLP. This seems to help in speeding up the submission rather than first predicting using the encoder then using the MLP. 

We use a Baysian Optimizer to find the optimal HPs for out model. 20 trials take about 2 hours on GPU.


In [ ]:
model_fn = lambda hp: build_model(
    hp, x_train.shape[-1], y_train.shape[-1], encoder)

In [ ]:
tuner = CVTuner(
    hypermodel = model_fn,
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('val_auc', direction='max'),
        num_initial_points=4,
        max_trials=20))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [ ]:
#collapse-output
gkf = PurgedGroupTimeSeriesSplit(n_splits = 5, group_gap=20)
splits = list(gkf.split(y_train, groups=train_df['date'].values))
tuner.search((x_train,),(y_train,),
             splits=splits,
             batch_size=4096,
             epochs=100,
             callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    'val_auc', mode='max', patience=3)])

In [ ]:
hp  = tuner.get_best_hyperparameters(1)[0]
pd.to_pickle(hp,f'./best_hp_{SEED}.pkl')
for fold, (train_indices, test_indices) in enumerate(splits):
    model = model_fn(hp)
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
    model.save_weights(f'./model_{SEED}_{fold}.hdf5')
    model.compile(Adam(hp.get('lr')/100),loss='binary_crossentropy')
    model.fit(X_test,y_test,epochs=3,batch_size=4096)
    model.save_weights(f'./model_{SEED}_{fold}_finetune.hdf5')
tuner.results_summary()